# Memória, Variáveis, Comunicação e Estado de um Programa

Há várias possíveis estratégias para usar a memória de um computador para se realizar a comunicação entre partes dele -- por exemplo, entre duas funções -- e para se manter o **estado atual** de execução de um programa. Dentre elas, podemos destacar o uso de variáveis locais, globais e parâmetros entre funções.

Vamos fazer aqui um exercício nesse contexto e você será conduzido a experimentar cada uma dessas estratégias, mesmo que em algumas ocasiões não seja a forma recomendada de se desenvolver um programa.


# Exercício do Empréstimo

Em um financiamento com juros compostos e número de parcelas fixas parte-se dos seguintes parâmetros:
* `S` - valor da primeira parcela
* `N` - número de parcelas
* `J` - percentual de juros mensal

A primeira parcela a ser paga do financiamento é sempre igual a `S`. A partir daí é feita uma atualização mensal da parcela, em que cada nova parcela é calculada a partir da parcela do mês anterior, conforme a fórmula:

> Parcela<sub>mês</sub> = Parcela<sub>mês-1</sub> * (1 + `J` / 100)

O financiamento encerra quando as `N` parcelas são pagas.

Exemplo:
* `S`: 200
* `N`: 5
* `J`: 1%

Parcelas do financiamento:
`200`; `202`; `204.02`; `206.06`; `208.12`

## Exercício Parte 1 - Escrevendo um Programa

Dado o problema descrito, escreva um programa que calcule as parcelas de um empréstimo para os seguintes valores:
* `S`: 200
* `N`: 5
* `J`: 1%

In [7]:
#include <stdio.h>

int main(){
    float S = 200;
    int N = 5;
    float J = 1;
    float proxima_parcela = S;
    for(int i = 0;i < N;i++){
        printf("parcela %d: %.2f\n",(i+1),proxima_parcela);
        proxima_parcela = proxima_parcela*(1 + (J/100));
    }
}

parcela 1: 200.00
parcela 2: 202.00
parcela 3: 204.02
parcela 4: 206.06
parcela 5: 208.12


## Exercício Parte 2 - Escrevendo um Módulo (Função)

Reescreva o código acima de forma que seu programa faça uso de uma função `proximaParcela` que seja responsável pelo cálculo de uma única parcela X do empréstimo. Utilize as boas práticas de modularização que você aprendeu, evitando dependências do módulo com o programa principal. A função deve apenas calcular uma única parcela, portanto, ficará a cargo da função principal que chama `proximaParcela` realizar o restante do processo, para que sejam apresentadas todas as parcelas do financiamento.

In [8]:
#include <stdio.h>

float proximaParcela(float S, float J){
    return(S*(1 + (J/100)));
}

int main(){
    float S = 200;
    int N = 5;
    float J = 1;
    for(int i = 0;i < N;i++){
        printf("parcela %d: %.2f\n",(i+1),S);
        S = proximaParcela(S,J);
    }
}

parcela 1: 200.00
parcela 2: 202.00
parcela 3: 204.02
parcela 4: 206.06
parcela 5: 208.12


## Exercício Parte 3 - Mantendo o Estado

Modifique a função (módulo) `proximaParcela` acima de modo que ela tenha uma memória da situação em que estão as coisas, sem que o `main` precise ficar informando novamente coisas que já informou no início. Para isso serão permitidas as seguintes modificações:

1. você pode desmembrar a função em mais de uma, por exemplo, uma você chama no começo do empréstimo e outra a cada parcela;

2. você pode usar técnicas de uso de variáveis não recomendadas que geram dependência dos módulos com o programa principal.

Você deve organizar o código de tal maneira que o `main` informe para as funções os dados do financiamento apenas uma única vez e depois possa solicitar o cálculo da parcela subsequente sem informar tudo novamente.

In [9]:
#include <stdio.h>

float S, J;
int N,i;

void configuraParcela(float S_configura,int N_configura,float J_configura){
    i = 0;
    S = S_configura;
    N = N_configura;
    J = J_configura;
}

void proximaParcela(){
    S = S*(1 + (J/100));
    i++;
}


int main(){
    configuraParcela(200,5,1);
    while(i < 5){
        printf("parcela %d: %.2f\n",(i+1),S);
        proximaParcela();
    }
}

parcela 1: 200.00
parcela 2: 202.00
parcela 3: 204.02
parcela 4: 206.06
parcela 5: 208.12


## Exercício Parte 4 - Minimizando os Parâmetros

Retomando a solução da `Parte 2`, em que a função `main` deve ter usado vários parâmetros para a comunicação com a função `proximaParcela`,  modifique a forma como você representa o empréstimo, de forma que a função `proximaParcela` receba sempre um único parâmetro capaz de representar o estado completo do empréstimo.

In [10]:
#include <stdio.h>

typedef struct{
    float S;
    float J;
    int N;
    int i;
}emprestimo;

void proximaParcela(emprestimo *Emprestimo){
    Emprestimo->i++;
    Emprestimo->S = Emprestimo->S*(1 + (Emprestimo->J/100));
}

int main(){
    emprestimo Emprestimo;
    Emprestimo.S = 200;
    Emprestimo.N = 5;
    Emprestimo.J = 1;
    Emprestimo.i = 0;
    while(Emprestimo.i < Emprestimo.N){
        printf("parcela %d: %.2f\n",(Emprestimo.i+1),Emprestimo.S);
        proximaParcela(&Emprestimo);
    }
}

parcela 1: 200.00
parcela 2: 202.00
parcela 3: 204.02
parcela 4: 206.06
parcela 5: 208.12


## Exercício Parte 5 - Múltiplos Empréstimos

Considere que há múltiplos empréstimos que podem ser controlados em paralelo. A sua função `main` deve ser capaz de apresentar no console as parcelas de mais de um empréstimo de modo paralelo, mantendo um controle para cada empréstimo separadamente. Você deve decidir qual das soluções tomará como ponto de partida, se deve modificar a função `main`, a função `proximaParcela` ou ambas para atender esse requisito da melhor forma possível.

Adote uma solução compacta e generalizável, de tal modo que, cada novo empréstimo só exija a informação dos parâmetros de partida, sem expansão do código.

Por exemplo, suponha os seguintes dois empréstimos em paralelo:

### Empréstimo 1
* `S`: 200
* `N`: 5
* `J`: 1%

### Empréstimo 2
* `S`: 500
* `N`: 7
* `J`: 2%

A saída esperada é:
~~~
Emprestimo 1: parcela 1 eh 200.00
Emprestimo 2: parcela 1 eh 500.00
Emprestimo 1: parcela 2 eh 202.00
Emprestimo 2: parcela 2 eh 510.00
Emprestimo 1: parcela 3 eh 204.02
Emprestimo 2: parcela 3 eh 520.20
Emprestimo 1: parcela 4 eh 206.06
Emprestimo 2: parcela 4 eh 530.60
Emprestimo 1: parcela 5 eh 208.12
Emprestimo 2: parcela 5 eh 541.22
Emprestimo 2: parcela 6 eh 552.04
Emprestimo 2: parcela 7 eh 563.08
~~~

O exemplo mostra dois empréstimos, mas a estrutura deve ser genérica o suficiente para controlar N empréstimos em paralelo.

In [11]:
#include <stdio.h>

typedef struct{
    float S;
    float J;
    int N;
    int i;
}emprestimo;

void proximaParcela(emprestimo *Emprestimo){
    Emprestimo->i++;
    Emprestimo->S = Emprestimo->S*(1 + (Emprestimo->J/100));
}

int main(){
    emprestimo Emprestimo_1;
    emprestimo Emprestimo_2;
    Emprestimo_1.S = 200;
    Emprestimo_1.N = 5;
    Emprestimo_1.J = 1;
    Emprestimo_1.i = 0;
    Emprestimo_2.S = 500;
    Emprestimo_2.N = 7;
    Emprestimo_2.J = 2;
    Emprestimo_2.i = 0;
    while(Emprestimo_1.i < Emprestimo_1.N || Emprestimo_2.i < Emprestimo_2.N){
        if(Emprestimo_1.i < Emprestimo_1.N){
            printf("Emprestimo 1 - parcela %d: %.2f\n",(Emprestimo_1.i+1),Emprestimo_1.S);
            proximaParcela(&Emprestimo_1);
        }
        if(Emprestimo_2.i < Emprestimo_2.N){
            printf("Emprestimo 2 - parcela %d: %.2f\n",(Emprestimo_2.i+1),Emprestimo_2.S);
            proximaParcela(&Emprestimo_2);
        }
    }
}

Emprestimo 1 - parcela 1: 200.00
Emprestimo 2 - parcela 1: 500.00
Emprestimo 1 - parcela 2: 202.00
Emprestimo 2 - parcela 2: 510.00
Emprestimo 1 - parcela 3: 204.02
Emprestimo 2 - parcela 3: 520.20
Emprestimo 1 - parcela 4: 206.06
Emprestimo 2 - parcela 4: 530.60
Emprestimo 1 - parcela 5: 208.12
Emprestimo 2 - parcela 5: 541.22
Emprestimo 2 - parcela 6: 552.04
Emprestimo 2 - parcela 7: 563.08
